In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
#ADD YOUR DIRECTORY HERE
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/utils')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/models')

from utils.analysis import *
from utils.getters import * 
from utils.plotters import * 
from utils.helpers import * 
from utils.config import * 
from models_config.model_config_GSP import *


## Getting the best model for RepKD

In [3]:
analysis_type="model_assessment"
dataset_split="val"
dataset="gender_data"
models_args=[
    gat_gat_student_lsp_ensamble_2_args,
    gat_gat_student_lsp_ensamble_3_args,
    gat_gat_student_lsp_ensamble_4_args,
    gat_gat_student_lsp_ensamble_5_args
]
views = [0,2,4,5]

#GET ALL OF THE MODEL BEST STUDENTS FOR ALL RUNS AND DIFFERENT PICKING CRITERIA FOR OUR MODEL 
all_model_metrics_mean, all_model_metrics_var, all_model_best_student = get_all_best_student(analysis_type, dataset_split, dataset, models_args, views)

gat_student_lsp_ensamble_2
[[0.9533333333333334, 0.00032222222222222277, 1], [0.9533333333333334, 0.00032222222222222277, 1], [0.9533333333333334, 0.00032222222222222277, 1], [0.9533333333333334, 0.00032222222222222277, 1], [0.9533333333333334, 0.00032222222222222277, 1]]
[[0.9586111111111111, 1.4969135802469005e-05, 1], [0.9586111111111111, 1.4969135802469005e-05, 1], [0.9586111111111111, 1.4969135802469005e-05, 1], [0.9586111111111111, 1.4969135802469005e-05, 1], [0.9586111111111111, 1.4969135802469005e-05, 1]]
[[0.9795833333333335, 0.00027239583333333445, 1], [0.9655555555555557, 0.0001685185185185181, 0], [0.9795833333333335, 0.00027239583333333445, 1], [0.9795833333333335, 0.00027239583333333445, 1], [0.9795833333333335, 0.00027239583333333445, 1]]
[[0.9755555555555556, 3.05555555555559e-05, 0], [0.9341666666666667, 0.0004969907407407426, 1], [0.9341666666666667, 0.0004969907407407426, 1], [0.9755555555555556, 3.05555555555559e-05, 0], [0.9755555555555556, 3.05555555555559e-05, 0]

In [84]:
df_rep, df_acuracy, df_var, index = final_student(all_model_metrics_mean, all_model_metrics_var, all_model_best_student, selection_method='weighted acc')

In [85]:
df_rep, df_acuracy, df_var, index

(          2         3         4         5
 0  0.953333  0.977222  1.000000  0.993333
 1  0.958611  0.965278  0.987222  0.993333
 2  0.979583  0.997500  1.000000  1.000000
 3  0.975556  0.966667  0.993472  1.000000
 4  0.966771  0.976667  0.995174  0.996667,
           2         3         4         5
 0  0.576902  0.560335  0.562831  0.577630
 1  0.646946  0.560711  0.570842  0.560335
 2  0.558988  0.560335  0.560335  0.560335
 3  0.560383  0.559904  0.560335  0.568870
 4  0.585805  0.560321  0.563586  0.566793,
           2             3             4             5
 0  0.003320  1.110223e-16  1.356280e-03  6.691485e-03
 1  0.004715  7.560841e-04  3.528424e-03  1.110223e-16
 2  0.000706  1.110223e-16  1.110223e-16  1.110223e-16
 3  0.000145  1.436782e-04  1.110223e-16  8.540286e-03
 4  0.002906  3.848073e-04  1.890057e-03  5.424769e-03,
      2    3    4    5
 0  1.0  0.0  1.0  4.0
 1  1.0  2.0  3.0  2.0
 2  1.0  2.0  1.0  1.0
 3  0.0  1.0  3.0  4.0)

## COMPARISON WITH OTHER METHODS

In [70]:
#GET THE REPRODUCIBILITY SCORE FOR ALL MODELS 
mean_all_runs, mean_all_std = reproducibility_mulitple_runs_student_specific(
    dataset="gender_data", 
    views=[0,2,4,5], 
    models=["gat", "gat_student", "gat_student_teacher", "fitnet_gat", "lsp_gat", "mskd_gat", "gat_student_lsp_ensamble_2"],
    CV=["3Fold", "5Fold", "10Fold"], 
    runs=[i for i in range(10)], 
    students=[
        [1,1,1,0]
    ], 
    model_args = [
        gat_args, 
        gat_gat_student_args, 
        gat_gat_student_args, 
        gat_gat_fitnet_student_args, 
        gat_gat_lsp_student_args, 
        gat_gat_mskd_student_args,
        gat_gat_student_lsp_ensamble_2_args
    ]
)

In [79]:
reproducibility_frame = pd.DataFrame(mean_all_runs.T)

In [80]:
models=["gat", "gat_student", "gat_student_teacher", "fitnet_gat", "lsp_gat", "mskd_gat", "repKD"]
reproducibility_frame.columns = models
reproducibility_frame = reproducibility_frame*100
reproducibility_frame["delta student"] = ((reproducibility_frame["repKD"] - reproducibility_frame["gat"]) / reproducibility_frame["gat"])*100
reproducibility_frame["delta teacher"] = ((reproducibility_frame["repKD"] - reproducibility_frame["gat_student"]) / reproducibility_frame["gat_student"])*100

reproducibility_frame["delta vanilla"] = ((reproducibility_frame["repKD"] - reproducibility_frame["gat_student_teacher"]) / reproducibility_frame["gat_student_teacher"])*100
reproducibility_frame["delta fitnet"] = ((reproducibility_frame["repKD"] - reproducibility_frame["fitnet_gat"]) / reproducibility_frame["fitnet_gat"])*100
reproducibility_frame["delta lsp"] = ((reproducibility_frame["repKD"] - reproducibility_frame["lsp_gat"]) / reproducibility_frame["lsp_gat"])*100
reproducibility_frame["delta mskd"] = ((reproducibility_frame["repKD"] - reproducibility_frame["mskd_gat"]) / reproducibility_frame["mskd_gat"])*100

reproducibility_frame.T

,0,1,2,3,4
gat,91.722222,93.333333,91.597222,92.416667,92.267361
gat_student,91.944444,90.736111,91.986111,91.750000,91.604167
gat_student_teacher,90.805556,90.277778,90.055556,91.208333,90.586806
fitnet_gat,78.000000,84.125000,74.486111,85.222222,80.458333
lsp_gat,88.986111,85.763889,82.208333,85.111111,85.517361
mskd_gat,88.097222,84.472222,81.972222,85.444444,84.996528
repKD,95.333333,95.861111,97.958333,97.555556,96.677083
delta student,3.937008,2.708333,6.944655,5.560565,4.779287
delta teacher,3.685801,5.648247,6.492526,6.327581,5.537867
delta vanilla,4.986234,6.184615,8.775447,6.959038,6.723140


In [83]:
reproducibility_frame_std = pd.DataFrame(mean_all_std.T)*100
models=["gat", "gat_student", "gat_student_teacher", "fitnet_gat", "lsp_gat", "mskd_gat", "repKD"]
reproducibility_frame_std.columns = models
reproducibility_frame_std.T

,0,1,2,3,4
gat,3.310221,3.361570,5.211295,4.064108,1.307332
gat_student,3.333912,4.073045,3.429206,3.744955,0.864548
gat_student_teacher,4.784233,4.769372,5.305556,3.371340,1.324410
fitnet_gat,14.752615,11.079303,14.531130,7.387845,6.483631
lsp_gat,6.747499,5.434533,8.755389,6.528310,4.760148
mskd_gat,7.366887,5.157923,9.198556,8.700211,5.204268
repKD,1.795055,0.386900,1.650442,0.552771,0.679714


In [116]:
views=[0, 2, 4, 5]
CV=["3Fold", "5Fold", "10Fold"]

runs = [i for i in range(10)]
dataset = "gender_data"
dataset_split="val"
analysis_type="model_assessment"
metrics = ["acc"]

final_model = ["gat_student_lsp_ensamble_2"]
final_model_args = [gat_gat_student_lsp_ensamble_2_args]
best_student = [1,1,1,0]
baseline_models = ["gat", "gat_student", "gat_student_teacher", "fitnet_gat", "lsp_gat", "mskd_gat"]
baseline_models_args = [
    gat_args, 
    gat_gat_student_args, 
    gat_gat_student_args, 
    gat_gat_fitnet_student_args, 
    gat_gat_lsp_student_args,
    gat_gat_mskd_student_args
]

In [117]:
average_across_views_df, average_across_views_df_std_mean = performance_mulitple_runs_student_specific(final_model, final_model_args, best_student, baseline_models, baseline_models_args, views, CV, runs, dataset, dataset_split, analysis_type, metrics)

In [118]:
average_across_views_df = pd.DataFrame(average_across_views_df)
models=["gat", "gat_student", "gat_student_teacher", "fitnet_gat", "lsp_gat", "mskd_gat", "repKD"]

average_across_views_df.columns = models
average_across_views_df = average_across_views_df*100

average_across_views_df["delta teacher"] = ((average_across_views_df["repKD"] - average_across_views_df["gat"]) / average_across_views_df["gat"])*100
average_across_views_df["delta student"] = ((average_across_views_df["repKD"] - average_across_views_df["gat_student"]) / average_across_views_df["gat_student"])*100
average_across_views_df["delta vanilla"] = ((average_across_views_df["repKD"] - average_across_views_df["gat_student_teacher"]) / average_across_views_df["gat_student_teacher"])*100
average_across_views_df["delta fitnet"] = ((average_across_views_df["repKD"] - average_across_views_df["fitnet_gat"]) / average_across_views_df["fitnet_gat"])*100
average_across_views_df["delta lsp"] = ((average_across_views_df["repKD"] - average_across_views_df["lsp_gat"]) / average_across_views_df["lsp_gat"])*100
average_across_views_df["delta mskd"] = ((average_across_views_df["repKD"] - average_across_views_df["mskd_gat"]) / average_across_views_df["mskd_gat"])*100

average_across_views_df.T

,0,1,2,3,4
gat,62.028917,67.774275,65.287638,66.396705,65.371884
gat_student,57.524683,61.165815,60.289983,59.962390,59.735718
gat_student_teacher,58.049584,63.116655,61.128468,60.654510,60.737304
fitnet_gat,62.520177,66.365191,65.852429,66.460628,65.299606
lsp_gat,63.275583,66.402727,66.667679,66.834138,65.795032
mskd_gat,63.447090,66.921493,66.926761,66.478758,65.943526
repKD,57.690150,64.694557,55.898792,56.038334,58.580458
delta teacher,-6.994748,-4.544081,-14.380741,-15.600732,-10.388909
delta student,0.287645,5.769142,-7.283450,-6.544197,-1.933951
delta vanilla,-0.619183,2.499977,-8.555222,-7.610608,-3.551106


In [119]:
average_across_views_df_std_mean = pd.DataFrame(average_across_views_df_std_mean)
models=["gat", "gat_student", "gat_student_teacher", "fitnet_gat", "lsp_gat", "mskd_gat", "repKD"]

average_across_views_df_std_mean.columns = models
average_across_views_df_std_mean.T

,0,1,2,3,4,5
gat,0.620289,0.677743,0.652876,0.663967,0.653719,0.653719
gat_student,0.575247,0.611658,0.602900,0.599624,0.597357,0.597357
gat_student_teacher,0.580496,0.631167,0.611285,0.606545,0.607373,0.607373
fitnet_gat,0.625202,0.663652,0.658524,0.664606,0.652996,0.652996
lsp_gat,0.632756,0.664027,0.666677,0.668341,0.657950,0.657950
mskd_gat,0.634471,0.669215,0.669268,0.664788,0.659435,0.659435
repKD,0.576902,0.646946,0.558988,0.560383,0.585805,0.585805
